<a href="https://colab.research.google.com/github/kyowoii23/system_quality_project/blob/main/notebooks/AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pycaret
!pip install jinja
!pip install markupsafe==2.0.1

In [ ]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
train_err = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/train_err_data.csv')
train_problem = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/train_problem_data.csv')
train_quality = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/train_quality_data.csv')

In [ ]:
train_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [13]:
train_err.errcode.value_counts()

1                     8906967
0                     2599123
connection timeout    1835600
B-A8002                897863
80                     334018
                       ...   
4329                        1
3765                        1
4105                        1
5779                        1
25999                       1
Name: errcode, Length: 2805, dtype: int64

In [24]:
train_problem.head()

,user_id,time
0,19224,20201102200000
1,23664,20201116140000
2,15166,20201114130000
3,12590,20201108210000
4,15932,20201103210000


In [25]:
train_quality.head()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


In [26]:
id_error = train_err[['user_id','errtype']].values
id_error

array([[10000,    15],
       [10000,    12],
       [10000,    11],
       ...,
       [24999,     4],
       [24999,     4],
       [24999,    15]])

In [27]:
error = np.zeros((15000,42))
for person_idx, err in tqdm(id_error):
  error[person_idx - 10000, err - 1] += 1

error

100%|██████████| 16554663/16554663 [00:44<00:00, 375930.12it/s]


array([[  0.,   0.,   8., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 113.,  56.,   1.],
       [  0.,   0.,   2., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  58.,   8.,   5.],
       [  0.,   0.,   0., ...,   6.,   0.,   0.],
       [  0.,   0.,   4., ...,   0.,   0.,   0.]])

In [23]:
problem = np.zeros(15000)
train_problem.user_id.unique()
problem[train_problem.user_id.unique()-10000] = 1
problem

array([0., 1., 0., ..., 1., 1., 0.])

In [28]:
train = pd.DataFrame(data=error)
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0


In [29]:
train['problem'] = problem
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,problem
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,1.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0,0.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0,1.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0,1.0


In [30]:
clf = setup(data = train, target = 'problem')
clf

,Description,Value
0,session_id,7542
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 43)"
5,Missing Values,False
6,Numeric Features,42
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='problem',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_stra...
                 ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                 ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                 ('cluster_all', 'passthrough'),
                 ('dummy', Dummify(target='problem')),
                 (

In [31]:
best_5 = compare_models(sort = 'Accuracy', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7762,0.8008,0.5048,0.7536,0.6044,0.4568,0.4747,0.348
gbc,Gradient Boosting Classifier,0.7746,0.8014,0.4632,0.7834,0.5819,0.4412,0.4699,2.105
et,Extra Trees Classifier,0.7730,0.7973,0.4919,0.7531,0.5949,0.4467,0.4664,1.832
rf,Random Forest Classifier,0.7716,0.7971,0.4744,0.7618,0.5845,0.4386,0.4621,1.855
ada,Ada Boost Classifier,0.7671,0.7902,0.4730,0.7481,0.5793,0.4294,0.4511,0.571
lr,Logistic Regression,0.7478,0.7373,0.3783,0.7553,0.5038,0.3591,0.3977,1.731
lda,Linear Discriminant Analysis,0.7423,0.7406,0.3215,0.7970,0.4580,0.3269,0.3850,0.094
ridge,Ridge Classifier,0.7421,0.0000,0.3165,0.8034,0.4538,0.3246,0.3851,0.027
qda,Quadratic Discriminant Analysis,0.7321,0.7370,0.3086,0.7548,0.4372,0.3008,0.3526,0.056
nb,Naive Bayes,0.7203,0.7382,0.2530,0.7638,0.3791,0.2539,0.3187,0.039


In [32]:
blended = blend_models(estimator_list = best_5, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7848,0.8068,0.4937,0.7923,0.6083,0.4707,0.4958
1,0.7805,0.8069,0.4972,0.7746,0.6056,0.4634,0.4853
2,0.7852,0.8222,0.4775,0.8115,0.6012,0.4675,0.4982
3,0.7776,0.8000,0.4705,0.7882,0.5893,0.4498,0.4780
4,0.7694,0.8018,0.4684,0.7585,0.5791,0.4323,0.4562
Mean,0.7795,0.8076,0.4815,0.7850,0.5967,0.4567,0.4827
Std,0.0058,0.0078,0.0119,0.0177,0.0109,0.0141,0.0151


In [33]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7883,0.8056,0.4986,0.7576,0.6014,0.4655,0.4844


In [34]:
final_model = finalize_model(blended)

In [36]:
test_err  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/test_err_data.csv')
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((14999,42))
for person_idx, err in tqdm(id_error):
    test_x[person_idx - 30000,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
test = pd.DataFrame(data=test_x)

100%|██████████| 16532648/16532648 [00:47<00:00, 350508.85it/s]


In [37]:
predictions = predict_model(final_model, data = test)

In [38]:
#pycaret에서는 score이 label을 맞출 확률이기때문에 output을 제출 양식에 맞게 바꿔줍니다
x = []
for i in range(len(predictions['Score'])):
  if predictions['Label'][i] =='1.0':
    x.append(predictions['Score'][i])
  else:
    x.append(1-predictions['Score'][i])

In [39]:
sample_submssion = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/sample_submission.csv')
sample_submssion['problem'] = x
sample_submssion.to_csv("AutoML.csv", index = False)

# 1

In [ ]:
import os
import random # Analysis
import datetime as dt # Analysis
import numpy as np # Analysis
import pandas as pd # Analysis

from tqdm import tqdm

In [ ]:
import warnings
warnings.filterwarnings(action='ignore') # ignore warnings

In [ ]:
# pd.options.display.max_columns=1000
# pd.options.display.max_rows=200
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
train_err = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/train_err_data.csv')
train_qual = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/train_quality_data.csv')
train_prob = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/train_problem_data.csv')

test_err = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/test_err_data.csv')
test_qual = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/test_quality_data.csv')

In [ ]:
tre = train_err.iloc[:4000000,:].copy()
tre_missing = tre.groupby(['user_id','time','fwver','errtype'])['errcode'].unique().to_frame()
tre_missing['errcode'] = tre_missing['errcode'].apply(lambda x : len(x))

ltwo = len(tre_missing[tre_missing['errcode'] != 1])
lone = len(tre_missing[tre_missing['errcode'] == 1])
print("동일한 특성을 갖는 Row안에 다른 Err Code가 발생할 확률 : %.3f%%"%(ltwo/lone * 100))

동일한 특성을 갖는 Row안에 다른 Err Code가 발생할 확률 : 0.039%


In [ ]:
train_err[(train_err['user_id'] == 13639) & (train_err['fwver'] == '04.33.1261') &
          (train_err['errtype'] == 5) & (train_err['time'] == 20201121191718)]

,user_id,time,model_nm,fwver,errtype,errcode
3825744,13639,20201121191718,model_2,04.33.1261,5,NaN
3825745,13639,20201121191718,model_2,04.33.1261,5,40013


In [ ]:
train_err['errcode'] = train_err['errcode'].fillna('40013')

In [ ]:
test_err[test_err['errcode'].isnull()]

,user_id,time,model_nm,fwver,errtype,errcode
937967,30820,20201115044317,model_2,04.33.1261,5,NaN
4038892,33681,20201103110259,model_2,04.33.1185,5,NaN
9486881,38991,20201127213838,model_2,04.33.1261,5,NaN
10425473,39894,20201128144712,model_1,04.16.3553,5,NaN


In [ ]:
te = test_err.iloc[:4000000,:].copy()
te_missing = te.groupby(['user_id','time','fwver','errtype'])['errcode'].unique().to_frame()
te_missing['errcode'] = te_missing['errcode'].apply(lambda x : len(x))

ltwo = len(te_missing[te_missing['errcode'] != 1])
lone = len(te_missing[te_missing['errcode'] == 1])
print("동일한 특성을 갖는 Row안에 다른 Err Code가 발생할 확률 : %.3f%%"%(ltwo/lone * 100))

동일한 특성을 갖는 Row안에 다른 Err Code가 발생할 확률 : 0.054%


In [ ]:
test_err.iloc[937967,5] = '40053'
test_err.iloc[4038892,5] = '40053'
test_err.iloc[9486881,5] = '40053'
test_err.iloc[10425473,5] = '-1010'

In [ ]:
missing_value = pd.Series([43262, train_err.time.mode()[0], train_err.model_nm.mode()[0],
                           train_err.fwver.mode()[0], train_err.errtype.mode()[0],
                           train_err.errcode.mode()[0]], index = test_err.columns)

In [ ]:
test_err = test_err.append(missing_value, ignore_index = True)

In [ ]:
def make_datetime(x): # datetime 데이터로 변환
        x = str(x)
        year  = int(x[:4])
        month = int(x[4:6])
        day   = int(x[6:8])
        hour  = int(x[8:10])
        minute  = int(x[10:12])
        sec  = int(x[12:])
        return dt.datetime(year, month, day, hour, minute, sec)

In [ ]:
train_err.time = train_err.time.apply(lambda x : make_datetime(x))

train_qual = train_qual.sort_values(['user_id','time']).reset_index(drop=True)
train_qual.time = train_qual.time.apply(lambda x : make_datetime(x))

train_prob = train_prob.sort_values(['user_id','time']).reset_index(drop=True)
train_prob.time = train_prob.time.apply(lambda x : make_datetime(x))

test_err.time = test_err.time.apply(lambda x : make_datetime(x))

test_qual.time = test_qual.time.apply(lambda x : make_datetime(x))